In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
block_size = 8
batch_size = 4
learning_rate = 3e-4
max_iter = 10000
eval_iter = 250

cuda


In [5]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [7]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [8]:
data = torch.tensor(encode(text),dtype = torch.long)
print(data[:100])

tensor([80,  0,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32,
        29,  1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1,
        26, 49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45,
        37,  0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,
         1, 36, 25, 38, 28,  1, 39, 30,  1, 39])


In [9]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split== "train" else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device),y.to(device)
    return x,y
x,y = get_batch("train")
print("inputs")
print(x)
print("targets")
print(y)
    

inputs
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1],
        [ 1, 73, 68,  1, 54, 65, 65,  1],
        [ 1, 60, 68,  9,  1, 59, 68, 71],
        [64, 58, 58, 69,  1, 73, 61, 62]], device='cuda:0')
targets
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1],
        [73, 68,  1, 54, 65, 65,  1, 73],
        [60, 68,  9,  1, 59, 68, 71,  1],
        [58, 58, 69,  1, 73, 61, 62, 72]], device='cuda:0')


In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when the target is",context,"target is",target)

when the target is tensor([80]) target is tensor(0)
when the target is tensor([80,  0]) target is tensor(1)
when the target is tensor([80,  0,  1]) target is tensor(1)
when the target is tensor([80,  0,  1,  1]) target is tensor(28)
when the target is tensor([80,  0,  1,  1, 28]) target is tensor(39)
when the target is tensor([80,  0,  1,  1, 28, 39]) target is tensor(42)
when the target is tensor([80,  0,  1,  1, 28, 39, 42]) target is tensor(39)
when the target is tensor([80,  0,  1,  1, 28, 39, 42, 39]) target is tensor(44)


In [12]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train","val"]:
        losses = torch.zeros(eval_iter)
        for k in range(eval_iter):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split]  = losses.mean()
    model.train()
    return out
        

In [13]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    def forward(self,index,targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits,loss
 
    def generate(self,index,max_new_tokens):
      for _ in range(max_new_tokens):
            # Step 1: Forward pass to get logits
            logits, loss = self.forward(index)

            # Step 2: Focus only on last time step
            logits = logits[:, -1, :]  # shape becomes (B, C)

            # Step 3: Convert logits to probabilities
            probs = F.softmax(logits, dim=-1)  # shape: (B, C)

            # Step 4: Sample from the probability distribution
            index_next = torch.multinomial(probs, num_samples=1)  # shape: (B, 1)

            # Step 5: Append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1)  # shape: (B, T+1
      return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)
context = torch.zeros((1,1),dtype=torch.long,device = device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


dlo XWt,tWyCyyyZ?O*LWrxzr)3(GK;rBw.m'['K64slPT_*d1EV-:X,&'Mu:

A6c ( Mafl9:
G,N,KxG!V0j.(ypOb3IqC3xCZUEzFfsI,j)'9_98kLCsg*?&[_7DoUubYk?Xg8)L2qtHcGDAK2LzaRvO"﻿v8y!MeAV52﻿h0;pyU*T:)p;-
ASU6;rl410YU_9v)H8Vo6X[5?HAhcX.G4uBYp!WFB_7Adfc4Zj.yHlf&[)'neK[jp]Udjn;l﻿7V?7JC9v! OAtgSw&5y9w,PYp]50g7
" 7j;f0-U6we8[]CiG3['PdZ(f,V&o﻿&ew
6﻿ZG4'X.7GX.w'07MeAi-LZ9grhpDBY&"]S!- oQd_t
6OV7jeaQLfs_LacKRLx1YUFw!-AL?29['5"!W,w,_Qm1Bu
dWekjeJ)a9rJo﻿h "e﻿*sRSSXdv! ZpjROvv4N_[ek!J2gx,_m9ROTIUl﻿j;-vR5Jc﻿d;-['_La]p[?eAXVKTs﻿


In [17]:
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)
for iter in range(max_iter):
    if iter % eval_iter == 0:
        losses = estimate_loss()
        print(f"step:{iter},losses : {losses}")
    xb,yb = get_batch('train')
    logits,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step:0,losses : {'train': tensor(4.9470), 'val': tensor(4.9353)}
step:250,losses : {'train': tensor(4.8815), 'val': tensor(4.8679)}
step:500,losses : {'train': tensor(4.8252), 'val': tensor(4.8056)}
step:750,losses : {'train': tensor(4.7523), 'val': tensor(4.7530)}
step:1000,losses : {'train': tensor(4.7013), 'val': tensor(4.7081)}
step:1250,losses : {'train': tensor(4.6370), 'val': tensor(4.6141)}
step:1500,losses : {'train': tensor(4.5947), 'val': tensor(4.5835)}
step:1750,losses : {'train': tensor(4.5342), 'val': tensor(4.5349)}
step:2000,losses : {'train': tensor(4.4723), 'val': tensor(4.4629)}
step:2250,losses : {'train': tensor(4.4125), 'val': tensor(4.3973)}
step:2500,losses : {'train': tensor(4.3581), 'val': tensor(4.3550)}
step:2750,losses : {'train': tensor(4.3255), 'val': tensor(4.3060)}
step:3000,losses : {'train': tensor(4.2628), 'val': tensor(4.2502)}
step:3250,losses : {'train': tensor(4.2062), 'val': tensor(4.2191)}
step:3500,losses : {'train': tensor(4.1507), 'val': te

In [14]:
context = torch.zeros((1,1),dtype=torch.long,device = device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


jXzXEL21tF?! o5oe Iw,N[QUdWelPshk?RBb)3guS!U7AzpIG)SoY.F3anj.&TG8)JKxub3kH;V3FyyrlCdW!H,jI[]"TWN9w
YGog;7b2T:x﻿
..qe*,'He
wDZj;7*91o8-J-4!fXX55)H]
TDV3d'OOulFAtVNpyl?LXxnyampA o8

AKVCYUd&_Q,qW8MEzpO6DZC,4zL"[ReUTAulX?!-poA tn]vuam4cL,(1W!&In9GSjHh[RV;V)mQvTjrh LZgJ5GhXxa&Wbq-4H5AB.IqzL44rHpyPTWekzz]sRuah4c)'[&-wtiRzGEM[,20H-q t1eU
(:'oJv9JqW﻿7j!UU
AY1dvvr0!WH6RV(
NfS 0*w4 P tzBhCn2J(jgrtu86q]﻿
lp0B8XeE5A3Npy&3cTo*
zX,0T:6oh[&RlEzp]ZMe2(ALZP3hosOmW".fbSt
.R! 3Qs4D7"Z?ekB.7?GV:4*o&8zvEUe*:
osVNNx
